# Dependency Installments

In [2]:
!pip install git+https://github.com/csebuetnlp/normalizer

Keyring is skipped due to an exception: 'keyring.backends'
  Cloning https://github.com/csebuetnlp/normalizer to /tmp/pip-req-build-m5byb7qd
  Running command git clone --filter=blob:none --quiet https://github.com/csebuetnlp/normalizer /tmp/pip-req-build-m5byb7qd
  Resolved https://github.com/csebuetnlp/normalizer to commit d80c3c484e1b80268f2b2dfaf7557fe65e34f321
  Preparing metadata (setup.py) ... done


In [16]:
import json
import pandas as pd
import numpy as np
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import Perceptron
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

# Data Preprocessing Core Functionalities

In [6]:
import json
from normalizer import normalize

filepath = os.path.join(".", "dev.txt")
output_file = os.path.join(".", "dev.jsonl")
with open(filepath) as fp:
    lines = fp.readlines()
    cnt = 1
    all_data = []
    tokens = []
    tags = []
    for line in lines:
        
        # print("Line {}: {}".format(cnt, line.strip()))
        line = line.strip()
        if line == "":
            all_data.append({'tokens': tokens, 'tags': tags})
            # print(tokens, tags)
            tokens = []
            tags = []
            continue
        # words = line
        # print(words)
        if len(line) == 1:
            token = line
        else:
            token = normalized_text = normalize(
                line,
                unicode_norm="NFKC",          # type of unicode normalization (default "NFKC")
                punct_replacement="",       # an optional string or callable for replacing the punctuations (default `None`, i.e. no replacement)
                url_replacement="",         # an optional string or callable for replacing the URLS (default `None`, i.e. no replacement)
                emoji_replacement="",       # an optional string or callable for replacing the emojis (default `None`, i.e. no replacement)
                apply_unicode_norm_last=True  # whether to apply the unicode normalization before or after rule based replacements (default True)        
            )
        
        # print(token)
        tokens.append(token)
        tags.append('O')
        cnt += 1
    all_data.append({'tokens': tokens, 'tags': tags})
    # print(all_data)
        
with open(output_file, 'w', encoding='utf-8') as json_file:
    for item in all_data:
        json.dump(item, json_file, ensure_ascii=False)
        json_file.write('\n')

In [8]:
def readfile(path):
    data = []
    with open(path, 'r') as reader:
        for line in reader:
            data.append(json.loads(line))
    return data

In [9]:
train_path = os.path.join(".", "dev.jsonl")
train_data = readfile(train_path)


In [23]:
train_data_tokens = []
train_data_tags = []
for data in train_data:
    train_data_tokens = train_data_tokens + data["tokens"]
    train_data_tags = train_data_tags+ data['tags']
print(train_data_tokens[:10])
print(train_data_tags[:10])

['নিকটতম', 'বিমানবন্দরটি', '(', '১৫৮', 'কিমি', ')', 'রাজীব', 'গান্ধী', 'আন্তর্জাতিক', 'বিমানবন্দর']
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


In [24]:
word_dict = {'tokens': train_data_tokens,
              'tags': train_data_tags}

df = pd.DataFrame.from_dict(word_dict)
df.head()

,tokens,tags
0,নিকটতম,O
1,বিমানবন্দরটি,O
2,(,O
3,১৫৮,O
4,কিমি,O


In [26]:
y = df.tags.values
X = df.drop('tags', axis=1)

In [28]:
X

,tokens
0,নিকটতম
1,বিমানবন্দরটি
2,(
3,১৫৮
4,কিমি
...,...
152437,বিজ্ঞাপনগুলি
152438,প্রচার
152439,করা
152440,শুরু


In [29]:
v = DictVectorizer()

In [30]:
X = v.fit_transform(X.to_dict('records'))

In [21]:
classes = np.unique(y)
classes = classes.tolist()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 1, random_state=0)
# X_train.shape, y_train.shape

In [ ]:
# from sklearn.svm import LinearSVC, NuSVC, SVC

In [ ]:
# svm = LinearSVC()

In [ ]:
# y_pred = svm.predict(X_test)

## Loading The Model

In [22]:
import pickle

loaded_model = pickle.load(open(r'model/finalized_model.sav', 'rb'))
result = loaded_model.predict(X_test)
print(result)

ValueError: X has 24584 features per sample; expecting 17380